In [5]:
s the latest version of bitsandbytes: `

SyntaxError: invalid syntax (3077241375.py, line 1)

In [8]:
pip install bitsandbytes==0.46.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.47.0
    Uninstalling bitsandbytes-0.47.0:
      Successfully uninstalled bitsandbytes-0.47.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
import requests
from PIL import Image

model = AutoModelForCausalLM.from_pretrained(
    "reubk/MolmoE-1B-0924-NF4",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,  # or 'auto'
    device_map="cpu",
    # quantization handled automatically if checkpoint is NF4
)

Created quantized layers based on config


In [ ]:
print(model)  # Look for "NF4Tensor" or "4-bit" in layer weights

# Or inspect a parameter:
for name, param in model.named_parameters():
    if "lm_head" in name or "layers.0" in name:
        print(f"{name}: dtype={param.dtype}, shape={param.shape}")
        break

In [13]:
from transformers import AutoTokenizer

# Load base tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/MolmoE-1B-0924', trust_remote_code=True)

# Manually add special image tokens (from reubk's tokenizer_config.json)
special_tokens = {
    "<image_patch>": 32000,
    "<image_col>": 32001,
    "<image_start>": 32002,
    "<image_end>": 32003
}

# Add tokens to tokenizer
tokenizer.add_tokens(list(special_tokens.keys()), special_tokens=True)

# Manually set token IDs (optional — if tokenizer assigned different IDs)
for token, token_id in special_tokens.items():
    tokenizer.added_tokens_encoder[token] = token_id
    tokenizer.added_tokens_decoder[token_id] = token

print("✅ Special tokens added:")
for token in special_tokens:
    print(f"  {token} → ID {tokenizer.convert_tokens_to_ids(token)}")

✅ Special tokens added:
  <image_patch> → ID 50285
  <image_col> → ID 50286
  <image_start> → ID 50287
  <image_end> → ID 50288


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoImageProcessor
from transformers import ProcessorMixin
import torch

# 1. Load base tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/MolmoE-1B-0924', trust_remote_code=True)

# 2. Inject special image tokens (from reubk's tokenizer_config.json)
special_tokens = {
    "<image_patch>": 32000,
    "<image_col>": 32001,
    "<image_start>": 32002,
    "<image_end>": 32003
}

tokenizer.add_tokens(list(special_tokens.keys()), special_tokens=True)

# Optional: Force token IDs (if needed)
for token, token_id in special_tokens.items():
    tokenizer.added_tokens_encoder[token] = token_id
    tokenizer.added_tokens_decoder[token_id] = token

print("✅ Tokenizer special tokens:")
for token in special_tokens:
    print(f"  {token} → ID {tokenizer.convert_tokens_to_ids(token)}")

# 3. Load image processor from community repo
image_processor = AutoImageProcessor.from_pretrained(
    'reubk/MolmoE-1B-0924-NF4',
    trust_remote_code=True
)

# 4. Load model
model = AutoModelForCausalLM.from_pretrained(
    'reubk/MolmoE-1B-0924-NF4',
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map='cpu'
)

# 5. Define fixed processor
class SimpleMolmoProcessor(ProcessorMixin):
    attributes = ["image_processor", "tokenizer"]
    image_processor_class = "AutoImageProcessor"
    tokenizer_class = "AutoTokenizer"

    def __init__(self, image_processor, tokenizer):
        super().__init__(image_processor, tokenizer)
        self.image_processor = image_processor
        self.tokenizer = tokenizer

    def __call__(self, images=None, text=None, **kwargs):
        inputs = {}
        
        # Get required special token IDs
        image_patch_token_id = self.tokenizer.convert_tokens_to_ids("<image_patch>")
        image_col_token_id = self.tokenizer.convert_tokens_to_ids("<image_col>")
        image_start_token_id = self.tokenizer.convert_tokens_to_ids("<image_start>")
        image_end_token_id = self.tokenizer.convert_tokens_to_ids("<image_end>")
    
        # Safety check
        required_tokens = {
            "<image_patch>": image_patch_token_id,
            "<image_col>": image_col_token_id,
            "<image_start>": image_start_token_id,
            "<image_end>": image_end_token_id,
        }
        for name, token_id in required_tokens.items():
            if token_id == self.tokenizer.unk_token_id:
                raise ValueError(f"Special token {name} not found in tokenizer!")
    
        if text:
            text_inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
            inputs.update(text_inputs)
        
        if images:
            # ✅ CONVERT PIL IMAGES TO NUMPY BEFORE PASSING
            if isinstance(images, list):
                images = [np.array(img) for img in images]
            else:
                images = np.array(images)
    
            image_inputs = self.image_processor(
                images,
                return_tensors="pt",
                image_patch_token_id=image_patch_token_id,
                image_col_token_id=image_col_token_id,
                image_start_token_id=image_start_token_id,
                image_end_token_id=image_end_token_id,
            )
            inputs.update(image_inputs)
        
        return inputs

    def process(self, images=None, text=None):
        return self(images=images, text=text)

# 6. Instantiate processor
processor = SimpleMolmoProcessor(image_processor, tokenizer)

print(f"✅ Model loaded on: {model.device}")
print(f"✅ Tokenizer is: {type(processor.tokenizer).__name__}")

✅ Tokenizer special tokens:
  <image_patch> → ID 50285
  <image_col> → ID 50286
  <image_start> → ID 50287
  <image_end> → ID 50288
Created quantized layers based on config
✅ Model loaded on: cpu
✅ Tokenizer is: GPTNeoXTokenizerFast


In [19]:
from PIL import Image
import numpy as np
from transformers import GenerationConfig

image = Image.open("345.png").convert("RGB")
print(f"✅ Loaded image: {image.size}, mode={image.mode}")

inputs = processor.process(images=[image], text="What time is shown?")
inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

output = model.generate_from_batch(
    inputs,
    GenerationConfig(max_new_tokens=50),
    tokenizer=processor.tokenizer
)

generated = processor.tokenizer.decode(
    output[0, inputs['input_ids'].shape[1]:], 
    skip_special_tokens=True
)

print("✅ Output:", generated.strip())

✅ Loaded image: (545, 545), mode=RGB


AttributeError: 'list' object has no attribute 'shape'